# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [0]:
import torch as th

In [0]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
y = x + x

In [0]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [0]:
!pip install --upgrade syft

     |████████████████████████████████| 225kB 2.2MB/s 
     |████████████████████████████████| 450kB 45.8MB/s 
     |████████████████████████████████| 256kB 44.6MB/s 
     |████████████████████████████████| 204kB 44.1MB/s 
     |████████████████████████████████| 389kB 45.0MB/s 
     |████████████████████████████████| 81kB 28.6MB/s 
     |████████████████████████████████| 1.4MB 42.2MB/s 
     |████████████████████████████████| 51kB 19.8MB/s 
     |████████████████████████████████| 276kB 46.0MB/s 
     |████████████████████████████████| 122kB 43.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
      Successfully uninstalled msgpack-0.5.6
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
   

In [0]:
import syft as sy

W0711 14:39:17.395351 139801093379968 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0711 14:39:17.416212 139801093379968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
hook = sy.TorchHook(th)

In [0]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [0]:
bob = sy.VirtualWorker(hook, id="bob")

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x = x.send(bob)

In [0]:
bob._objects

{783865760: tensor([1, 2, 3, 4, 5])}

In [0]:
x.location

<VirtualWorker id:bob #objects:1>

In [0]:
x.id_at_location

783865760

In [0]:
x.id

42561321233

In [0]:
x.owner

<VirtualWorker id:me #objects:0>

In [0]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [0]:
x

(Wrapper)>[PointerTensor | me:42561321233 -> bob:783865760]

In [0]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [0]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [0]:
bob=sy.VirtualWorker(hook,id="bob")

In [0]:
alice=sy.VirtualWorker(hook,id="alice")

In [0]:
x=th.randint(1,10,[10])
x

tensor([2, 2, 3, 4, 6, 2, 7, 1, 6, 7])

In [0]:
x_ptr=x.send(bob,alice)

In [0]:
# x_ptr=x_ptr.float().mean()
alice

<VirtualWorker id:alice #objects:4>

In [0]:
x_ptr.get(sum_results=True)

tensor(8.)

AttributeError: ignored

# Lesson: Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [0]:
x

(Wrapper)>[PointerTensor | me:18024113559 -> bob:1597465589]

In [0]:
y

(Wrapper)>[PointerTensor | me:29766135883 -> bob:31412341160]

In [0]:
z = x + y

In [0]:
z

(Wrapper)>[PointerTensor | me:27724196846 -> bob:6123685564]

In [0]:
z = z.get()
z

tensor([ 1,  4,  9, 16, 25])

In [0]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:41713350601 -> bob:54554229783]

In [0]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()

In [0]:
z.backward()

(Wrapper)>[PointerTensor | me:72134898360 -> bob:84082081156]

In [0]:
x = x.get()

In [0]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [0]:
x.grad

tensor(20., requires_grad=True)

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [0]:
import torch.nn.functional as F
input=th.tensor([[1.,1],[1,0],[0,1],[0,0]]).send(bob)
label=th.tensor([[1.],[1],[0],[0]]).send(bob)



(Wrapper)>[PointerTensor | me:76680720125 -> bob:65497709904]::data

In [0]:
w1=th.randn([2,6], requires_grad=True).send(bob)
b1=th.randn([1,6], requires_grad=True).send(bob)



In [0]:
w2=th.randn([6,1], requires_grad=True).send(bob)
b2=th.randn([1,1], requires_grad=True).send(bob)


In [0]:
def sigmoid(x):
  return 1/(1+th.exp(-x))

In [0]:
for _ in range(2000):
  h1=sigmoid(th.mm(input,w1)+b1)


  out=sigmoid(th.mm(h1,w2)+b2)
  
  loss=F.binary_cross_entropy(out, label)
  loss.backward()
  w1.data.sub_(w1.grad*0.2)
  b1.data.sub_(b1.grad*0.2)
  w2.data.sub_(w2.grad*0.2)
  b2.data.sub_(b2.grad*0.2)
  w1.grad*=0
  b1.grad*=0
  w2.grad*=0
  b2.grad*=0
  print(loss.get().data)

tensor(0.0139)
tensor(0.0139)
tensor(0.0138)
tensor(0.0138)
tensor(0.0138)
tensor(0.0138)
tensor(0.0138)
tensor(0.0137)
tensor(0.0137)
tensor(0.0137)
tensor(0.0137)
tensor(0.0137)
tensor(0.0137)
tensor(0.0136)
tensor(0.0136)
tensor(0.0136)
tensor(0.0136)
tensor(0.0136)
tensor(0.0135)
tensor(0.0135)
tensor(0.0135)
tensor(0.0135)
tensor(0.0135)
tensor(0.0134)
tensor(0.0134)
tensor(0.0134)
tensor(0.0134)
tensor(0.0134)
tensor(0.0134)
tensor(0.0133)
tensor(0.0133)
tensor(0.0133)
tensor(0.0133)
tensor(0.0133)
tensor(0.0132)
tensor(0.0132)
tensor(0.0132)
tensor(0.0132)
tensor(0.0132)
tensor(0.0132)
tensor(0.0131)
tensor(0.0131)
tensor(0.0131)
tensor(0.0131)
tensor(0.0131)
tensor(0.0130)
tensor(0.0130)
tensor(0.0130)
tensor(0.0130)
tensor(0.0130)
tensor(0.0130)
tensor(0.0129)
tensor(0.0129)
tensor(0.0129)
tensor(0.0129)
tensor(0.0129)
tensor(0.0129)
tensor(0.0128)
tensor(0.0128)
tensor(0.0128)
tensor(0.0128)
tensor(0.0128)
tensor(0.0128)
tensor(0.0127)
tensor(0.0127)
tensor(0.0127)
tensor(0.0

In [0]:
out=out.get()

In [0]:
out

tensor([[0.9856],
        [0.9885],
        [0.0117],
        [0.0176]], requires_grad=True)

In [0]:
w1.grad

tensor([[-2.0825e-03, -3.1833e-04, -1.2322e-04],
        [ 2.9329e-03,  9.2790e-05, -2.7978e-05],
        [-1.3097e-03, -2.7794e-04,  8.3252e-05],
        [ 1.6909e-03,  4.8462e-05, -1.0258e-04],
        [-1.5993e-03,  7.4773e-05, -1.7332e-04]])

In [0]:
out=out.get()


In [0]:
out

tensor([[0.9966],
        [0.9972],
        [0.0029],
        [0.0041]], requires_grad=True)

In [0]:
x=x.get()
x

tensor([[-0.6516,  0.6162,  1.0106,  1.3454,  0.2684],
        [ 1.3804,  0.9218,  3.0091, -0.5988, -1.3014],
        [ 0.8171, -1.1758, -0.1974,  0.0266,  0.5463],
        [ 0.1135, -0.1423,  1.8862,  0.3520, -0.1075],
        [ 0.1909, -0.3648, -0.3256,  0.7250, -0.5396],
        [ 1.0933, -1.2085, -0.6667, -0.0858,  0.6897],
        [ 0.0122,  1.4891, -0.2990, -1.0622,  0.6830],
        [-2.4573, -1.1542,  0.2709, -1.5584,  0.6325],
        [ 0.9898,  0.1380, -1.9752, -1.5378,  1.2027],
        [-0.2116,  0.7338, -0.9320, -0.1794, -0.1203]])

# Lesson: Garbage Collection and Common Errors


In [0]:
bob = bob.clear_objects()

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{76095677946: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{21999285274: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x

(Wrapper)>[PointerTensor | me:3420727541 -> bob:44765379635]

In [0]:
bob._objects

{44765379635: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [0]:
bob._objects

{44765379635: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [0]:
bob._objects

{44765379635: tensor([1, 2, 3, 4, 5])}

In [0]:
bob = bob.clear_objects()
bob._objects

{}

In [0]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{14018917294: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [0]:
z = x + y

TensorsNotCollocatedException: ignored

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [0]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:46419059800 -> bob:14412738960]
Tensor B: tensor([1, 1, 1, 1, 1])

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [0]:
from torch import nn, optim
import torch as th

In [0]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# A Toy Model
model = nn.Linear(2,1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [0]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(0.0860)
tensor(0.0586)
tensor(0.0402)
tensor(0.0278)
tensor(0.0194)
tensor(0.0136)
tensor(0.0096)
tensor(0.0069)
tensor(0.0049)
tensor(0.0036)
tensor(0.0026)
tensor(0.0019)
tensor(0.0014)
tensor(0.0010)
tensor(0.0008)
tensor(0.0006)
tensor(0.0004)
tensor(0.0003)
tensor(0.0002)
tensor(0.0002)


In [0]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [0]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [0]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [0]:
bob

<VirtualWorker id:bob #objects:5>

In [0]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()
            model.move(bob)
            model.weight.grad
            # get smarter model back
            model = model.get()

            print(loss.get())

In [0]:

train()

tensor(0.6236, requires_grad=True)
tensor(0.4768, requires_grad=True)
tensor(0.2001, requires_grad=True)
tensor(0.2951, requires_grad=True)
tensor(0.1111, requires_grad=True)
tensor(0.1768, requires_grad=True)
tensor(0.0648, requires_grad=True)
tensor(0.1062, requires_grad=True)
tensor(0.0381, requires_grad=True)
tensor(0.0642, requires_grad=True)
tensor(0.0225, requires_grad=True)
tensor(0.0390, requires_grad=True)
tensor(0.0134, requires_grad=True)
tensor(0.0239, requires_grad=True)
tensor(0.0081, requires_grad=True)
tensor(0.0148, requires_grad=True)
tensor(0.0049, requires_grad=True)
tensor(0.0093, requires_grad=True)
tensor(0.0030, requires_grad=True)
tensor(0.0058, requires_grad=True)
tensor(0.0019, requires_grad=True)
tensor(0.0037, requires_grad=True)
tensor(0.0012, requires_grad=True)
tensor(0.0024, requires_grad=True)
tensor(0.0008, requires_grad=True)
tensor(0.0016, requires_grad=True)
tensor(0.0005, requires_grad=True)
tensor(0.0010, requires_grad=True)
tensor(0.0004, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [0]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

<VirtualWorker id:alice #tensors:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x = x.send(alice)

In [0]:
bob._objects

{60919291280: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{46793677354: (Wrapper)>[PointerTensor | alice:46793677354 -> bob:60919291280]}

In [0]:
y = x + x

In [0]:
y

(Wrapper)>[PointerTensor | me:92538866869 -> alice:32871421246]

In [0]:
bob._objects

{60919291280: tensor([1, 2, 3, 4, 5]),
 72123043838: tensor([ 2,  4,  6,  8, 10])}

In [0]:
alice._objects

{32871421246: (Wrapper)>[PointerTensor | alice:32871421246 -> bob:72123043838],
 46793677354: (Wrapper)>[PointerTensor | alice:46793677354 -> bob:60919291280]}

In [0]:
jon = sy.VirtualWorker(hook, id="jon")

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [0]:
bob._objects

{18160533154: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{53783886866: (Wrapper)>[PointerTensor | alice:53783886866 -> bob:18160533154]}

In [0]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:53783886866 -> bob:18160533154]

In [0]:
bob._objects

{18160533154: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{}

In [0]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [0]:
bob._objects

{}

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [0]:
bob._objects

{51514801886: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{80795951581: (Wrapper)>[PointerTensor | alice:80795951581 -> bob:51514801886]}

In [0]:
del x

In [0]:
bob._objects

{}

In [0]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [0]:
bob.clear_objects()
alice.clear_objects()
jon.clear_objects()

<VirtualWorker id:jon #objects:0>

In [0]:
x = torch.tensor([1,2,3,4,5]).send(bob)

<VirtualWorker id:bob #objects:3>

In [0]:
alice._objects

{}

In [0]:
x.move(alice)

(Wrapper)>[PointerTensor | me:54954175478 -> alice:54954175478]

In [0]:
bob._objects

{}

In [0]:
alice._objects

{54954175478: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice).send(jon)

In [0]:
bob._objects

{92897231289: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{19877040920: (Wrapper)>[PointerTensor | alice:19877040920 -> bob:92897231289]}

In [0]:
jon._objects

{17246609480: (Wrapper)>[PointerTensor | jon:17246609480 -> alice:19877040920]}

In [0]:
x

(Wrapper)>[PointerTensor | me:87878937621 -> jon:17246609480]

In [0]:
x.remote_get()

(Wrapper)>[PointerTensor | me:87878937621 -> jon:17246609480]

In [0]:
bob._objects

{}

In [0]:
alice._objects

{}

In [0]:
jon._objects

{17246609480: (Wrapper)>[PointerTensor | jon:17246609480 -> bob:92897231289]}

In [0]:
x.move(bob)

(Wrapper)>[PointerTensor | me:87205391815 -> bob:87205391815]

In [0]:
x

(Wrapper)>[PointerTensor | me:87205391815 -> bob:87205391815]

In [0]:
bob._objects

{87205391815: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{94092707138: tensor([1, 2, 3, 4, 5])}

In [1]:
!pip install tf-encrypted

! URL="https://github.com/openmined/PySyft.git" && FOLDER="PySyft" && if [ ! -d $FOLDER ]; then git clone -b dev --single-branch $URL; else (cd $FOLDER && git pull $URL && cd ..); fi;

!cd PySyft; python setup.py install  > /dev/null

import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
!pip install --upgrade --force-reinstall lz4
!pip install --upgrade --force-reinstall websocket
!pip install --upgrade --force-reinstall websockets
!pip install --upgrade --force-reinstall zstd

     |████████████████████████████████| 2.1MB 2.8MB/s 
     |████████████████████████████████| 276kB 37.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Cloning into 'PySyft'...
remote: Enumerating objects: 28675, done.
remote: Total 28675 (delta 0), reused 0 (delta 0), pack-reused 28675
Receiving objects: 100% (28675/28675), 32.07 MiB | 19.74 MiB/s, done.
Resolving deltas: 100% (19010/19010), done.
zip_safe flag not set; analyzing archive contents...
zip_safe flag not set; analyzing archive contents...
__pycache__.zstd.cpython-36: module references __file__
     |████████████████████████████████| 389kB 2.8MB/s 
ERROR: syft 0.1.21a1 has requirement msgpack>=0.6.1, but you'll have msgpack 0.5.6 which is incompatible.
ERROR: syft 0.1.21a1 has requirement tf_encrypted!=0.5.7,>=0.5.4, b

In [21]:
!pip install pytorch==1.0.1

  ERROR: Could not find a version that satisfies the requirement pytorch==1.0.1 (from versions: 0.1.2, 1.0.2)
ERROR: No matching distribution found for pytorch==1.0.1


In [0]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import syft as sy

In [23]:

hook = sy.TorchHook(torch)  
bob = sy.VirtualWorker(hook, id="bob")  
alice = sy.VirtualWorker(hook, id="alice")
secure_worker=sy.VirtualWorker(hook, id="secure_worker")
bob.clear_objects()
alice.clear_objects()
secure_worker.clear_objects()

W0722 09:28:38.812662 140642623997824 hook.py:98] Torch was already hooked... skipping hooking process


<VirtualWorker id:secure_worker #objects:0>

In [0]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 10
        self.save_model = True

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")


kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [31]:

federated_train_loader = sy.FederatedDataLoader(
                          datasets.FashionMNIST('../data', train=True, download=True,
                          transform=transforms.Compose([
                          transforms.ToTensor(),
                          transforms.Normalize((0.1307,), (0.3081,))])).federate((bob, alice)),
                          batch_size=args.batch_size, shuffle=True,**kwargs)
train_loader = torch.utils.data.DataLoader(
                          datasets.FashionMNIST('../data', train=True, download=True,
                          transform=transforms.Compose([
                          transforms.ToTensor(),
                          transforms.Normalize((0.1307,), (0.3081,))])),
                          batch_size=args.batch_size, shuffle=True,**kwargs)

test_loader = torch.utils.data.DataLoader(
                       datasets.FashionMNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))])),
                       batch_size=args.test_batch_size, shuffle=True,**kwargs)

W0722 09:35:17.093801 140642623997824 dataloader.py:197] The following options are not supported: num_workers: 1, pin_memory: True


In [0]:
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [0]:
# def train(args, model, device, train_loader, optimizer, epoch):
#     model.train()
#     for batch_idx, (data, target) in enumerate(federated_train_loader):
#         model.send(data.location) # <-- NEW: send the model to the right location
#         data, target = data.to(device), target.to(device)
#         optimizer.zero_grad()
#         output = model(data)
#         loss = F.nll_loss(output, target)
#         loss.backward()
#         optimizer.step()
#         model.get() # <-- NEW: get the model back
#         if batch_idx % args.log_interval == 0:
#             loss = loss.get() # <-- NEW: get the loss back
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.item()))
            
            

In [0]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
model=Net().to(device)

optimizer=optim.SGD(model.parameters(), lr=args.lr)


In [36]:
data,target=next(iter(train_loader))
data, target = data.to(device), target.to(device)
optimizer.zero_grad()
output = model(data)
loss = F.nll_loss(output, target)
loss.backward()
optimizer.step()
test(args, model, device, test_loader)

RuntimeError: ignored

In [0]:
model_bob= model.copy().send(bob)
model_alice=model.copy().send(alice)
optimizer_bob = optim.SGD(model_bob.parameters(), lr=args.lr)
optimizer_alice = optim.SGD(model_alice.parameters(), lr=args.lr)

In [29]:
model_bob.train()
model_alice.train()
data,target=next(iter(federated_train_loader))
if data.location == bob:
          
  model_bob.send(data.location) # <-- NEW: send the model to the right location
  data, target = data.to(device), target.to(device)
  optimizer_bob.zero_grad()
  output_bob = model_bob(data)
  loss_bob = F.nll_loss(output_bob, target)
  loss_bob.backward()
  optimizer_bob.step()
  loss_bob=loss_bob.get().data
  test(args, model_bob, device, test_loader)
elif data.location==alice:
          
  model_alice.send(data.location) # <-- NEW: send the model to the right location
  data, target = data.to(device), target.to(device)
  optimizer_alice.zero_grad()
  output_alice = model_alice(data)
  loss_alice = F.nll_loss(output_alice, target)
  loss_alice.backward()
  optimizer_alice.step()
  loss_alice=loss_alice.get().data
  test(args, model_bob, device, test_loader)

RuntimeError: ignored

In [18]:

for epoch in range(1, args.epochs + 1):
    model_bob.train()
    model_alice.train()
    for data, target in federated_train_loader:
      if data.location == bob:
          
          model_bob.send(data.location) # <-- NEW: send the model to the right location
          data, target = data.to(device), target.to(device)
          optimizer_bob.zero_grad()
          output_bob = model_bob(data)
          loss_bob = F.nll_loss(output_bob, target)
          loss_bob.backward()
          optimizer_bob.step()
          loss_bob=loss_bob.get().data
          test(args, model_bob, device, test_loader)
      elif data.location==alice:
          
          model_alice.send(data.location) # <-- NEW: send the model to the right location
          data, target = data.to(device), target.to(device)
          optimizer_alice.zero_grad()
          output_alice = model_alice(data)
          loss_alice = F.nll_loss(output_alice, target)
          loss_alice.backward()
          optimizer_alice.step()
          loss_alice=loss_alice.get().data
          test(args, model_bob, device, test_loader)
    model_alice.move(secure_worker)
    model_bob.move(secure_worker)
    with torch.no_grad():
        model.weight.set_(((model_alice.conv1.weight.data + model_bob.conv1.weight.data) / 2).get())
        model.bias.set_(((model_alice.conv1.bias.data + model_bob.conv1.bias.data) / 2).get())    
        model.weight.set_(((model_alice.conv2.weight.data + model_bob.conv2.weight.data) / 2).get())
        model.bias.set_(((model_alice.conv2.bias.data + model_bob.conv2.bias.data) / 2).get())    
        model.weight.set_(((model_alice.fc1.weight.data + model_bob.fc1.weight.data) / 2).get())
        model.bias.set_(((model_alice.fc1.bias.data + model_bob.fc1.bias.data) / 2).get())    
        model.weight.set_(((model_alice.fc2.weight.data + model_bob.fc2.weight.data) / 2).get())
        model.bias.set_(((model_alice.fc2.bias.data + model_bob.fc2.bias.data) / 2).get())    

        print('Train Epoch: {} \tAlice Loss: {:.6f} \bob Loss: {:.6f}'.format(
                epoch, loss_alice.item(),loss_bob.item()))
            
            
  
    

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

RuntimeError: ignored